In [10]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio.Blast.Applications import NcbiblastxCommandline
from Bio.Blast.Applications import NcbimakeblastdbCommandline

In [24]:
import os

def get_fasta_files(directory):
    fasta_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".fasta"):
                fasta_files.append(os.path.join(root, file))
    return fasta_files

def read_fasta(file_path):
    sequences = {}
    with open(file_path, 'r') as file:
        sequence_id = None
        sequence = ''
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if sequence_id:
                    sequences[sequence_id] = sequence
                sequence_id = line[1:]
                sequence = ''
            else:
                sequence += line
        if sequence_id:
            sequences[sequence_id] = sequence
    return sequences

def write_fasta(sequences, output_file):
    with open(output_file, 'w') as file:
        for sequence_id, sequence in sequences.items():
            file.write(f'>{sequence_id}\n')
            file.write(f'{sequence}\n')

def merge_fasta_files(input_directory, output_file):
    input_files = get_fasta_files(input_directory)
    merged_sequences = {}
    for input_file in input_files:
        sequences = read_fasta(input_file)
        merged_sequences.update(sequences)
    write_fasta(merged_sequences, output_file)

# Usage
input_directory = '../lib'  # Directory containing FASTA files
output_file = 'merged.fasta'  # Output file name
merge_fasta_files(input_directory, output_file)




In [25]:
cline = NcbimakeblastdbCommandline(dbtype="nucl",input_file="merged.fasta", input_type="fasta", out="/Users/david/Desktop/web-align/backend/utils")

In [26]:
cline

NcbimakeblastdbCommandline(cmd='makeblastdb', out='/Users/david/Desktop/web-align/backend/utils', dbtype='nucl', input_file='merged.fasta', input_type='fasta')

In [8]:
fasta_string = open("m_cold.fasta").read()
result_handle = NCBIWWW.qblast("blastn", "nt", fasta_string)
blast_record = NCBIXML.read(result_handle)

In [14]:
fasta_string

">gi|8332116|gb|BE037100.1|BE037100 MP14H09 MP Mesembryanthemum crystallinum cDNA 5' similar to cold acclimation protein, mRNA sequence\nCACTAGTACTCGAGCGTNCTGCACCAATTCGGCACGAGCAAGTGACTACGTTNTGTGAACAGAAAATGGG\nGAGAGAAATGAAGTACTTGGCCATGAAAACTGATCAATTGGCCGTGGCTAATATGATCGATTCCGATATC\nAATGAGCTTAAAATGGCAACAATGAGGCTCATCAATGATGCTAGTATGCTCGGTCATTACGGGTTTGGCA\nCTCATTTCCTCAAATGGCTCGCCTGCCTTGCGGCTATTTACTTGTTGATATTGGATCGAACAAACTGGAG\nAACCAACATGCTCACGTCACTTTTAGTCCCTTACATATTCCTCAGTCTTCCATCCGGGCCATTTCATCTG\nTTCAGAGGCGAGGTCGGGAAATGGATTGCCATCATTGCAGTCGTGTTAAGGCTGTTCTTCAACCGGCATT\nTCCCAGTTTGGCTGGAAATGCCTGGATCGTTGATACTCCTCCTGGTGGTGGCACCAGACTTCTTTACACA\nCAAAGTGAAGGAGAGCTGGATCGGAATTGCAATTATGATAGCGATAGGGTGTCACCTGATGCAAGAACAT\nATCAGAGCCACTGGTGGCTTTTGGAATTCCTTCACACAGAGCCACGGAACTTTTAACACAATTGGGCTTA\nTCCTTCTACTGGCTTACCCTGTCTGTTTATGGTCATCTTCATGATGTAGTAGCTTAGTCTTGATCCTAAT\nCCTCAAATNTACTTTTCCAGCTCTTTCGACGCTCTTGCTAAAGCCCATTCAATTCGCCCCATATTTCGCA\nCACATTCATTTCACCACCCAATACGTGCTCTCCTTCTCCCTCTCTCCCTCTCCTCCCTCTTTTCTTCCTC\

In [9]:
# Iterate over the alignments
for alignment in blast_record.alignments:
    print("**** Alignment ****")
    print("Accession:", alignment.accession)
    print("Length:", alignment.length)
    print("E-value:", alignment.hsps[0].expect)
    
    # Iterate over high-scoring pairs (HSPs)
    for hsp in alignment.hsps:
        print("Query start:", hsp.query_start)
        print("Query end:", hsp.query_end)
        print("Subject start:", hsp.sbjct_start)
        print("Subject end:", hsp.sbjct_end)
        print("Alignment:", hsp.align_length)
        print("Identity:", hsp.identities)
        print("Gaps:", hsp.gaps)
        print("Query sequence:", hsp.query)
        print("Subject sequence:", hsp.sbjct)

# Close the result handle
result_handle.close()

**** Alignment ****
Accession: XM_021875076
Length: 1173
E-value: 9.8612e-117
Query start: 59
Query end: 678
Subject start: 278
Subject end: 901
Alignment: 624
Identity: 473
Gaps: 4
Query sequence: ACAGAAAATGGGGAGAGAAATGAAGTACTTGGCCATGAAAACTGATCAATTGGCCGTGGCTAATATGATCGATTCCGATATCAATGAGCTTAAAATGGCAACAATGAGGCTCATCAATGATGCTAGTATGCTCGGT--CATTACGGG-TTTGGCACTCATTTCCTCAAATGGCTCGCCTGCCTTGCGGCTATTTACTTGTTGATATTGGATCGAACAAACTGGAGAACCAACATGCTCACGTCACTTTTAGTCCCTTACATATTCCTCAGTCTTCCATCCGGGCCATTTCATCTGTTCAGAGGCGAGGTCGGGAAATGGATTGCCATCATTGCAGTCGTGTTAAGGCTGTTCTTCAACCGGCATTTCCCAGTTTGGCTGGAAATGCCTGGATCGTTGATACTCCTCCTGGTGGTGGCACCAGACTTCTTTACACACAAAGTGAAGGAGAGCTGGATCGGAATTGCAATTATGATAGCGATAGGGTGTCACCTGATGCAAGAACATATCAGAGCCACTGGTGGCTTTTGGAATTCCTTCACACAGAGCCACGGAACTTTTAACACAATTGGGCTTATCCTTCTACTGGCTTACCCTGTCT-GTTTATGGTCATCTTCATGATGTA
Subject sequence: ACCGAAAATGGGCAGAGGAGTGAATTATATGGCAATGACACCTGAGCAACTAGCCGCGGCCAATTTGATCAACTCCGACATCAATGAGCTCAAGATCGTTGTGATGACACTCATTCATGATGCTTCTAGACTCGGCGGCACCTCAGGATTTGGAACTCA

In [3]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

def find_proteins(query_sequence, database_sequences):
    results = []
    
    for file_name in database_sequences:
        with open(file_name, "r") as handle:
            for record in SeqIO.parse(handle, "genbank"):
                database_sequence = record.seq
                alignments = pairwise2.align.localxx(query_sequence, database_sequence, one_alignment_only=True)
                
                if alignments:
                    result = {
                        'record_id': record.id,
                        'alignment': format_alignment(*alignments[0]),
                    }
                    results.append(result)

    return results

# Example: Load the query sequence
query_sequence = Seq("ATGCATGCATGCATGCA")

# Example: Load the database sequences
# database_sequences = ["NC_000852.gb", "NC_007346.gb", "NC_008724.gb", "NC_009899.gb", "NC_014637.gb", "NC_020104.gb", "NC_023423.gb", "NC_023640.gb", "NC_023719.gb", "NC_027867.gb"]
database_sequences = ["/Users/david/Desktop/web-dna-alignment/mysite/lib/NC_000852.gb"]
# Perform the search
search_results = find_proteins(query_sequence, database_sequences)

# Print the results
for result in search_results:
    print(f"Record ID: {result['record_id']}")
    print(f"Alignment:\n{result['alignment']}")
    print("\n" + "=" * 50 + "\n")


: 

In [ ]:
nt_viruses

In [1]:
from Bio import pairwise2
from Bio import SeqIO

/Users/david/opt/anaconda3/envs/bio/lib/python3.12/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [22]:
import os

def get_fasta_files(directory):
    fasta_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".fasta"):
                fasta_files.append(os.path.join(root, file))
    return fasta_files

input_directory = 'lib/coding_seqs'  # Directory containing FASTA files

genome_files = get_fasta_files(input_directory)

In [23]:
genome_files

['lib/coding_seqs/NC_007346.fasta',
 'lib/coding_seqs/NC_008724.fasta',
 'lib/coding_seqs/NC_023640.fasta',
 'lib/coding_seqs/NC_027867.fasta',
 'lib/coding_seqs/NC_014637.fasta',
 'lib/coding_seqs/NC_009899.fasta',
 'lib/coding_seqs/NC_000852.fasta',
 'lib/coding_seqs/NC_023719.fasta',
 'lib/coding_seqs/NC_020104.fasta',
 'lib/coding_seqs/NC_023423.fasta']

In [7]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

def fasta_to_seq_records(fasta_files):
    seq_records = []
    for file in fasta_files:
        with open(file, "r") as fasta_file:
            for record in SeqIO.parse(fasta_file, "fasta"):
                seq_records.append(record)
    return seq_records

genome_records = fasta_to_seq_records(genome_files)

In [8]:
genome_records

[SeqRecord(seq=Seq('TATATTTAACGCGAATGATTTAAGGATTTTTATGGTTTTAACCAAAACTCTGTA...TAT'), id='NC_007346.1', name='NC_007346.1', description='NC_007346.1 Emiliania huxleyi virus 86, complete genome', dbxrefs=[]),
 SeqRecord(seq=Seq('TTGCTCTGTCTGCTTGCATTTTGAAGATTTGCCATTGTCCCGACATTTCGTTGG...TCC'), id='NC_008724.1', name='NC_008724.1', description='NC_008724.1 Acanthocystis turfacea Chlorella virus 1, complete genome', dbxrefs=[]),
 SeqRecord(seq=Seq('AAAGTATATTATAGTTTATTTTATATATTACCTGTAGATGCGACGGAAATTAAT...TAA'), id='NC_016072.1', name='NC_016072.1', description='NC_016072.1 Megavirus chiliensis, complete genome', dbxrefs=[]),
 SeqRecord(seq=Seq('CTCGGCCGCGTCTCTAGGCCACCGTCGCGCCACCCCGTCGTTCCGTGCCACCAC...GAG'), id='NC_027867.1', name='NC_027867.1', description='NC_027867.1 Mollivirus sibericum isolate P1084-T, complete genome', dbxrefs=[]),
 SeqRecord(seq=Seq('TTAAATGTGTTAAACTTTATAGGTAAACATTCTTTCTGTCATCATTCACTATAA...ATT'), id='NC_014637.1', name='NC_014637.1', description='NC_014637.1 Cafeteria r

In [ ]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Align.Applications import ClustalOmegaCommandline
from Bio.Align import AlignInfo

def align_sequence_to_genomes(sequence, genome_files):
    alignments = []
    for genome_file in genome_files:
        # Align the input sequence to the current genome
        alignment_file = f"{genome_file.stem}_alignment.fasta"
        clustalo_cmd = ClustalOmegaCommandline(infile=genome_file, outfile=alignment_file, seq=sequence)
        clustalo_cmd()
        
        # Parse the alignment file
        alignment = AlignIO.read(alignment_file, "fasta")
        alignments.append(alignment)
        
        # If any alignments were found, return the first one
        if alignments:
            return alignments[0]

    # If no alignments were found, return None
    return None

# Example usage
sequence = "ATCGATCGATCG"
first_alignment = align_sequence_to_genomes(sequence, genome_files)
print(first_alignment)


In [17]:
from Bio import Align
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

def align_sequence_to_genomes(sequence, genome_records):
    aligner = Align.PairwiseAligner()
    for record in genome_records:
        alignment = aligner.align(sequence, record.seq)
        if alignment:
            return alignment[0]

    return None

# Example usage
sequence = "ATACTTACCTGACAGGGGAGGCACCATGATCACACAGGTGGTCCTCCCAGGGCGAGGCTCTTCCATTGCACTGCGGGAGGGTTGACCCCTGCGATTTCCCCAAATGTGGGAAACTCGACTGTATAATTTGTGGTAGTGGGGGACTGCGTTCGCGCTATCCCCCGATACTTACCTGACAGGGGAGGCACCATGATCACACAGGTGGTCCTCCCAGGGCGAGGCTCTTCCATTGCACTGCGGGAGGGTTGACCCCTGCGATTTCCCCAAATGTGGGAAACTCGACTGTATAATTTGTGGTAGTGGGGGACTGCGTTCGCGCTATCCCCCG"
# genome_records = [SeqRecord(Seq("GATCGATCGATC")), SeqRecord(Seq("ATCGATCGGCTA")), SeqRecord(Seq("ATCGGCTAGCTA"))]

alignment = align_sequence_to_genomes(sequence, genome_records)
print(alignment.sequences)


OverflowError: number of optimal alignments is larger than 9223372036854775807

In [24]:
import os

def get_fasta_files(directory):
    fasta_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".fasta"):
                fasta_files.append(os.path.join(root, file))
    return fasta_files

input_directory = 'lib/coding_seqs'  # Directory containing FASTA files

genome_files = get_fasta_files(input_directory)

def merge_fasta_files(input_files, output_file):
    with open(output_file, 'w') as out_fasta:
        for input_file in input_files:
            with open(input_file, 'r') as in_fasta:
                for line in in_fasta:
                    out_fasta.write(line)

# Example usage
# input_files = ['file1.fasta', 'file2.fasta', 'file3.fasta']
output_file = 'merged.fasta'
merge_fasta_files(genome_files, output_file)

In [26]:
from Bio import SeqIO
from Bio import Align

def align_sequence_to_fasta(sequence, fasta_file):
    aligner = Align.PairwiseAligner()
    with open(fasta_file, 'r') as fasta:
        for record in SeqIO.parse(fasta, "fasta"):
            record_sequence = str(record.seq)
            alignment = aligner.align(sequence, record_sequence)
            if alignment:
                return record.description, record_sequence, alignment
    return None

# Example usage
sequence = "ATCGATCGATCG"
fasta_file = "lib/merged.fasta"
header, record_sequence, alignment = align_sequence_to_fasta(sequence, fasta_file)
if alignment:
    print("Header:", header)
    print("Sequence:", record_sequence)
    print("Alignment:", alignment)
else:
    print("No alignment found.")

Header: lcl|NC_007346.1_cds_YP_293755.1_1 [locus_tag=EhV001] [db_xref=UniProtKB/TrEMBL:Q4A3C0] [protein=hypothetical protein] [protein_id=YP_293755.1] [location=complement(276..1022)] [gbkey=CDS]
Sequence: ATGTTATTTATTAAACATAACATCTCCGTATATTATACTACATCATACCATAAGTTAAGTCTCGTTATTGATATGTCTTTTGAGGAGTATAAGAAATATGCCGAAGATGTACTTAACGTAAAACAACATGCATTTTCATCGGTACTCATGATGGTACCTGTAGGCAAGTCTGGACATGTAGCAGAGCCTCCAGCAAAAGGCCTACCAACCATGCCAGACGAAGCTGATAATATTATTAGTCAGAAAACACCAGAGCTTAGGCGTTTGTACAAGAAAGCATGTTTATCGTGCTTAAGTGATGAGTTATATCGCGAACGATTTTTTGGGGTTTATTGTAAAGCATGCCGGAATGGAAAAGATCGAGTTGTTGCACACATGAAACGAACAGCAGAAGACAGGTATCATAAATTATTCCTTGCACGGAATAATGTTGAACAATCAAAAAAAAAGTGCAAAACATGCCCATTATATGCAAGTAGTACTGGATTTTGCAAAGAATGCAAACGACAAGACAATAATATCCGATACAACTTAAAGAGGAATACTCCATTTCCAGTTGACGGTAAATGTCATATATGCAAAAAGATTCCAAATAAAACATTAAATCTTGATCATGATCACATAAAGAAAACTGCAAGGGGGTGGGTATGTTTTCATTGTAATATATTGATTGGATTTATTGAAACTAACAATGTTACACGTCACAAAATTGGTGCATACTTTGACTATCTCGATAGAAATGCATAA
Alignment: <Bio.Align.PairwiseAlignments objec